In [1]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

Using TensorFlow backend.


In [2]:
english = []
french = []

In [3]:
english = open('english.txt', encoding='utf-8').read().split('\n')

In [4]:
french = open('french.txt', encoding='latin-1').read().split('\n')

In [5]:
num_lines = 0
with open('english.txt','r') as f:
    for line in f:
        num_lines +=1

In [6]:
num_lines

137860

In [7]:
num_line = 0
with open('english.txt','r') as f:
    for line in f:
        num_line +=1

In [8]:
num_line

137860

In [9]:
type(english)

list

In [10]:
import helper
# import problem_unittests as tests

source_path = 'english.txt'
target_path = 'french.txt'

english_chars = set()
french_chars = set()

In [11]:
sample_size = 1000

In [12]:
english_texts = []
target_texts = []
with open(source_path, 'r', encoding='utf-8') as f:
    input_lines = f.read().split('\n')
for input_line in input_lines[: min(sample_size, len(input_lines) - 1)]:
    english_text = input_line.split()
    english_texts.append(english_text)
    for char in english_text:
        if char not in english_chars:
            english_chars.add(char)


In [13]:
with open(target_path, 'r', encoding='latin-1') as f:
    target_lines = f.read().split('\n')
for target_line in target_lines[: min(sample_size, len(target_lines) - 1)]:
    target_text = target_line.split()
    target_texts.append(target_text)
    for char in target_text:
        if char not in french_chars:
            french_chars.add(char)

In [14]:
english_chars= sorted(list(english_chars))
french_chars = sorted(list(french_chars))
input_token_index = dict([(char, i) for i, char in enumerate(english_chars)])
target_token_index = dict([(char, i) for i, char in enumerate(french_chars)])

In [15]:
target_token_index

{',': 0,
 '-': 1,
 '.': 2,
 '?': 3,
 'a': 4,
 'agréable': 5,
 'aimait': 6,
 'aime': 7,
 'aiment': 8,
 'aimez': 9,
 'aimons': 10,
 'aimé': 11,
 'aimé.': 12,
 'aimée': 13,
 'aimés': 14,
 'aller': 15,
 'allons': 16,
 'allé': 17,
 'amusant': 18,
 'animal': 19,
 'animaux': 20,
 'août': 21,
 'au': 22,
 'automne': 23,
 'automobile': 24,
 'aux': 25,
 'avril': 26,
 'banane': 27,
 'bananes': 28,
 'beau': 29,
 'belle': 30,
 'blanc': 31,
 'blanche': 32,
 'bleu': 33,
 'bleue': 34,
 'brillant': 35,
 'brillante': 36,
 'california': 37,
 'californie': 38,
 'calme': 39,
 'camion': 40,
 'ce': 41,
 'cet': 42,
 'cette': 43,
 'chat': 44,
 'chaud': 45,
 'chaude': 46,
 'chaux': 47,
 'cher': 48,
 'cheval': 49,
 'chevaux': 50,
 'chien': 51,
 'chiens': 52,
 'chine': 53,
 'chinois': 54,
 'citron': 55,
 'citrons': 56,
 'clémentes': 57,
 'comme': 58,
 'comment': 59,
 'conduisait': 60,
 'conduit': 61,
 'cours': 62,
 "d'": 63,
 'dans': 64,
 'de': 65,
 'dernier': 66,
 'des': 67,
 'difficile': 68,
 'doux': 69,
 'décem

In [16]:
encoder_input_data = np.zeros((len(english_texts), max([len(txt) for txt in english_texts]), len(english_chars)),
    dtype='float32')
decoder_input_data = np.zeros((len(english_texts),  max([len(txt) for txt in target_texts]), len(french_chars)),
    dtype='float32')
decoder_target_data = np.zeros((len(english_texts),  max([len(txt) for txt in target_texts]), len(french_chars)),
    dtype='float32')

for i, (english_text, target_text) in enumerate(zip(english_texts, target_texts)):
    for t, char in enumerate(english_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text): 
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1

In [17]:
encoder_inputs = Input(shape=(None, len(english_chars)))
encoder = LSTM(256 , return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

encoder_states = [state_h, state_c]

In [18]:
decoder_inputs = Input(shape=(None,len(french_chars)))

decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,initial_state=encoder_states)
decoder_dense = Dense(len(french_chars), activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [21]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,batch_size=64,epochs=200,validation_split=0.2)

Train on 800 samples, validate on 200 samples
Epoch 1/200
800/800 [==============================] - 2s 2ms/step - loss: 0.5743 - val_loss: 0.6905
Epoch 2/200
800/800 [==============================] - 1s 983us/step - loss: 0.5143 - val_loss: 0.6871
Epoch 3/200
800/800 [==============================] - 1s 982us/step - loss: 0.5075 - val_loss: 0.7120
Epoch 4/200
800/800 [==============================] - 1s 982us/step - loss: 0.4951 - val_loss: 0.6918
Epoch 5/200
800/800 [==============================] - 1s 985us/step - loss: 0.4894 - val_loss: 0.6859
Epoch 6/200
800/800 [==============================] - 1s 983us/step - loss: 0.4916 - val_loss: 0.7012
Epoch 7/200
800/800 [==============================] - 1s 981us/step - loss: 0.4702 - val_loss: 0.6828
Epoch 8/200
800/800 [==============================] - 1s 985us/step - loss: 0.4694 - val_loss: 0.6813
Epoch 9/200
800/800 [==============================] - 1s 986us/step - loss: 0.4588 - val_loss: 0.6858
Epoch 10/200
800/800 [=======

800/800 [==============================] - 1s 984us/step - loss: 0.0442 - val_loss: 0.8136
Epoch 80/200
800/800 [==============================] - 1s 986us/step - loss: 0.0418 - val_loss: 0.8148
Epoch 81/200
800/800 [==============================] - 1s 989us/step - loss: 0.0467 - val_loss: 0.7963
Epoch 82/200
800/800 [==============================] - 1s 985us/step - loss: 0.0425 - val_loss: 0.8338
Epoch 83/200
800/800 [==============================] - 1s 985us/step - loss: 0.0376 - val_loss: 0.7974
Epoch 84/200
800/800 [==============================] - 1s 987us/step - loss: 0.0378 - val_loss: 0.8244
Epoch 85/200
800/800 [==============================] - 1s 986us/step - loss: 0.0354 - val_loss: 0.8218
Epoch 86/200
800/800 [==============================] - 1s 986us/step - loss: 0.0325 - val_loss: 0.8526
Epoch 87/200
800/800 [==============================] - 1s 987us/step - loss: 0.0441 - val_loss: 0.8112
Epoch 88/200
800/800 [==============================] - 1s 986us/step - loss:

800/800 [==============================] - 1s 985us/step - loss: 0.0078 - val_loss: 1.0512
Epoch 158/200
800/800 [==============================] - 1s 986us/step - loss: 0.0246 - val_loss: 0.9723
Epoch 159/200
800/800 [==============================] - 1s 989us/step - loss: 0.0045 - val_loss: 0.9863
Epoch 160/200
800/800 [==============================] - 1s 987us/step - loss: 0.0056 - val_loss: 0.9825
Epoch 161/200
800/800 [==============================] - 1s 987us/step - loss: 0.0061 - val_loss: 0.9825
Epoch 162/200
800/800 [==============================] - 1s 988us/step - loss: 0.0172 - val_loss: 0.9858
Epoch 163/200
800/800 [==============================] - 1s 985us/step - loss: 0.0081 - val_loss: 0.9825
Epoch 164/200
800/800 [==============================] - 1s 987us/step - loss: 0.0043 - val_loss: 0.9745
Epoch 165/200
800/800 [==============================] - 1s 986us/step - loss: 0.0043 - val_loss: 0.9944
Epoch 166/200
800/800 [==============================] - 1s 983us/ste

In [22]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [23]:
def decode_seq(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1, len(french_chars)))
    target_seq[0, 0,0] = 1.

    stop_condition = False
    translated_sequence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        translated_sequence += sampled_char

     
        if (sampled_char == '\n' or
           len(translated_sequence) >  max([len(txt) for txt in target_texts])):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1,len(french_chars)))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return translated_sequence

In [24]:
for index in range(10):
    input_seq = encoder_input_data[index: index + 1]
    translated_sequence = decode_seq(input_seq)

    print('English text:', english_texts[index])
    print('Translated in French:', translated_sequence)

English text: ['new', 'jersey', 'is', 'sometimes', 'quiet', 'during', 'autumn', ',', 'and', 'it', 'is', 'snowy', 'in', 'april', '.']
Translated in French: estparfoiscalmependant
English text: ['the', 'united', 'states', 'is', 'usually', 'chilly', 'during', 'july', ',', 'and', 'it', 'is', 'usually', 'freezing', 'in', 'november', '.']
Translated in French: estgénéralementgénéralement
English text: ['california', 'is', 'usually', 'quiet', 'during', 'march', ',', 'and', 'it', 'is', 'usually', 'hot', 'in', 'june', '.']
Translated in French: estgénéralementcalmeau
English text: ['the', 'united', 'states', 'is', 'sometimes', 'mild', 'during', 'june', ',', 'and', 'it', 'is', 'cold', 'in', 'september', '.']
Translated in French: estparfoisparfoischaud
English text: ['your', 'least', 'liked', 'fruit', 'is', 'the', 'grape', ',', 'but', 'my', 'least', 'liked', 'is', 'the', 'apple', '.']
Translated in French: moinsaiméfruitestleraisin
English text: ['his', 'favorite', 'fruit', 'is', 'the', 'orange'